# Analysing a 1D double well using TRAM

Import deeptime and other preliminaries...

In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from deeptime.clustering import KMeans
from deeptime.markov.msm import TRAM, TRAMDataset

## The system
We define the potential to be a simple double well.

In [ ]:
def potential(x):
    return 10 * (x - 1) ** 4 * (x + 1) ** 4

Plot the potential over the range of interest

In [ ]:
x_min = -1.5
x_max = 1.5

xs = np.linspace(x_min, x_max, num=100) # We will sample 100 discrete bins
plt.plot(xs, potential(xs));

## Sampling

The following is a simple Markov-chain Monte Carlo (MCMC) sampling algorithm that samples our potential `U`.

In [ ]:
# this is a simple markov-chain monte carlo (MCMC) sampler that samples a trajectory
# in an n-dimensional space given a potential function U.
def sample_MCMC(U, d, beta=1.0, n_steps=1000, max_stepsize=1.):
    trajectory = np.zeros((n_steps, d), dtype=np.float32)
    p = lambda u: np.exp(-beta * u)
    r_prev = np.random.uniform(0, 8, size=d)

    for n in range(n_steps):

        r = r_prev + 2 * max_stepsize * (np.random.uniform(size=d)) - max_stepsize

        delta = U(r) - U(r_prev)
        if delta > 0:
            if p(delta) < np.random.uniform():
                r = r_prev
            else:
                r_prev = r
        else:
            r_prev = r

        trajectory[n] = r
    return trajectory

### An example trajectory
We sample an example trajectory. As we will very likely see in the histogram, we get stuck in one of the wells, and don't cross the barrier.

In [ ]:
traj = sample_MCMC(potential, 1, n_steps=1000)

plt.hist(traj, bins=10, density=True, color='C1');
plt.plot(xs, potential(xs))
plt.xlim(x_min, x_max);

### Sampling at multiple temperatures
To solve this sampling problem we will sample at multiple temperatures. We define six thermodynamic states by their temperatures. The unnormalized probabilities for each temperature are plotted. As we kan see, the higher the temperature, the more flattened the shape of the probability distribution becomes.

In [ ]:
temperatures = [1, 2, 3, 4, 10, 20, 30, 40, 50]  # six temperatures to perform parallel tempering.

for T in temperatures:
    plt.plot(xs, np.exp(-(1/T) * potential(xs)))

Now we will sample a trajectory for each of the the temperatures.

In [ ]:
N_SAMPLES_PER_TRAJ = 10000
trajectories = []

for T in temperatures:
    print(f"sampling for T={T} ...")
    traj = sample_MCMC(potential, 1, beta=1/T, n_steps=N_SAMPLES_PER_TRAJ, max_stepsize=2.)
    trajectories.append(traj.squeeze())
print('done!')

We can plot a histogram of all trajectories to see the distribution we samples. By sampling at multiple temperatures, we were now able to sample in the transition region.

In [ ]:
plt.hist(np.concatenate(trajectories), bins=100, density=True, color='C1',
         label='Histogram over visited states');

## Analysing the data with TRAM
Now we want recombine our data to obtain the original potential function. We do this by analysing our data using TRAM. 

### Bias matrices
First, we construct the bias energy matrices by computing the bias potential for each sample in each state. The bias energy for a temperature biased simulation is given by $b^k(x) = (\beta^k - \beta^0)\; U^0(x)$. In this example, we assume all quantities are unitless.

When we print the shape of one of the bias matrices we see it has shape `(N, S)`, `N` being the number of samples, and `S` the number of thermodynamic states. Thus, for each sample we have computed the energy of that sample in _all_ states.

In [ ]:
bias_matrices = []

betas = np.asarray(temperatures, dtype=float)**-1
bias_factors = betas - betas[0]

for traj in trajectories:
    energies = potential(traj)
    bias_potentials = energies[:, None] * bias_factors
    bias_matrices.append(bias_potentials)
    
print(bias_matrices[0].shape)

### The discrete trajectories
Now we discretize our samples into 10 Markov states along the x-axis. The discretized trajectories (`dtrajs`) are what allow `TRAM` to compute state counts and transition counts.

In [ ]:
# define the markov states
n_bins = 10
markov_states = np.linspace(x_min, x_max, n_bins, endpoint=True)
print(markov_states)

In [ ]:
# construct the discretized trajectories
dtrajs = [np.digitize(traj, markov_states, right=False) for traj in trajectories]

print(trajectories[0][:10])
print(dtrajs[0][:10])

### Applying TRAM
Now, we are ready to analyse the data using TRAM. 

In [ ]:
# first construct the dataset...
dataset = TRAMDataset(dtrajs=dtrajs, bias_matrices=bias_matrices, lagtime=10)

# ... and use that run TRAM 
tram = TRAM(maxiter=1000, progress=tqdm, callback_interval=3)
model = tram.fit_fetch(dataset)

We can have a look at the free energies per thermodynamic state and per Markov state (i.e. the $f_i^k$) that we estimated

In [ ]:
plt.contourf(model.biased_conf_energies);

And plot the free energies per Markov state, $f_i$, which already give us an impression of the actual shape of our potential:

In [ ]:
plt.plot(np.linspace(-1.5, 1.5, len(model.markov_state_energies)), model.markov_state_energies)

The underlying MEMM is a property of the model, namely the `msm_collection`. This contains all transition matrices for each thermodynamic state.

In [ ]:
MEMM = model.msm_collection
MEMM.transition_matrix[1,2]

The MEMM behaves as an MSM, namely the MSM of the currently selected thermodynamic state. By default, state 0 is selected. We can select the Markov model for state $k$ from the MSM collection by calling `select`:

In [ ]:
MEMM.select(2)
MEMM.transition_matrix[1,2]

## Estimating the PMF
Now we want to recover the potential of mean force (PMF). To do this, we define a number of bins (in this case 25) that we want to calculate the unbiased PMF over. We essentially calculate a probability distribution over all bins, and then take the negative logarithm to recover the PMF. 

In [ ]:
n_bins = 25
bins = np.linspace(-1.5, 1.5, n_bins)

# the bin number for each sample. Samples are binned into 25 bins
bin_numbers = [np.digitize(traj, bins, right=True) for traj in trajectories]

# pass samples to the model with their corresponding bin numbers
pmf = model.compute_PMF(dtrajs, bias_matrices, bin_numbers, n_bins = n_bins)
plt.plot(bins, pmf - pmf.min())

# compare with the original distribution, shifted so that the minimum lies at zero.
plt.plot(bins, potential(bins) - potential(bins).min(), 'k--');